In [13]:
import math
import pulp as p
from random import shuffle
import numpy as np


def negative(arg):
    temp = map(lambda x: np.array(x)*(-1), arg)
    return list(map(tuple, temp))


def convertIndex(arg):
    dictionary = {'a': 0, 'b': 1, 'c': 2, 'd':3, 'e':4, 'f':5, 'g':6, 'h':7}
    return ( 8 - arg[1], dictionary[arg[0]] )

def convertIndexBack(arg):
    dictionary = {0:'a', 1:'b', 2:'c', 3:'d', 4:'e', 5:'f', 6:'g', 7:'h'}
    return dictionary[arg[1]] + str(8 - arg[0])

def createBoard():
    grid = []
    for i in range(8):
        row = []
        for j in range(8):
            row.append('-')
        grid.append(row)
    return grid


# basis moves
e_1 = (1, 2)
e_2 = (1, -2)

# other moves
v_3 = (-1, 2)
v_4 = (-1, -2)
v_5 = (2, 1)
v_6 = (2, -1)
v_7 = (-2, 1)
v_8 = (-2, -1)

moves = [e_1, e_2, v_3, v_4, v_5, v_6, v_7, v_8]

temp = input("Enter starting square: ")
start = convertIndex( [temp[0], int(temp[1])] )

temp = input("Enter final square: ")
end = convertIndex( [temp[0], int(temp[1])] )


    
def printGrid(board):
    for i, row in enumerate(board):
        print('\n' + str(8 - i) + "    " + "    ".join( str(e) for e in row) )
    print("\n     a    b    c    d    e    f    g    h")
    print('\n\n')

def checkValid(p, v):
    if 0 <=(p[0] + v[0]) < 8 and 0 <= (p[1] + v[1]) < 8:
        return True
    else:
        return False

def integerMoves(**kwargs):
    # Create a LP Minimization problem 
    problem = p.LpProblem('Problem', p.LpMinimize) 

    displacement = kwargs['d'][0] - kwargs['s'][0], kwargs['d'][1] - kwargs['s'][1]
    
    # Create problem Variables 
    x1 = p.LpVariable("x1", lowBound = 0, upBound = None, cat='Integer') # Create a variable x1 >= 0 
    x2 = p.LpVariable("x2", lowBound = 0, upBound = None, cat='Integer') # Create a variable x2 >= 0 
    x3 = p.LpVariable("x3", lowBound = 0, upBound = None, cat='Integer') # Create a variable x3 >= 0 
    x4 = p.LpVariable("x4", lowBound = 0, upBound = None, cat='Integer') # Create a variable x4 >= 0 
    x5 = p.LpVariable("x5", lowBound = 0, upBound = None, cat='Integer') # Create a variable x5 >= 0 
    x6 = p.LpVariable("x6", lowBound = 0, upBound = None, cat='Integer') # Create a variable x6 >= 0 
    x7 = p.LpVariable("x7", lowBound = 0, upBound = None, cat='Integer') # Create a variable x7 >= 0 
    x8 = p.LpVariable("x8", lowBound = 0, upBound = None, cat='Integer') # Create a variable x8 >= 0 
    # Objective Function 
    problem +=  x1+x2+x3+x4+x5+x6+x7+x8
    # Constraints: 
    for i in range(2):
        k = 1 - i
        problem += kwargs['v1'][i] * x1 + kwargs['v2'][i] * x2  + kwargs['v3'][i] * x3 + kwargs['v4'][i]*x4 \
        + kwargs['v5'][i] * x5 + kwargs['v6'][i] * x6 + kwargs['v7'][i] * x7 + kwargs['v8'][i] * x8 == displacement[k]
    status = problem.solve() # Solver
    #print("It takes " + str(int(p.value(problem.objective))) + " moves")
    #print(p.LpStatus[status]) # The solution status 
    return [int(p.value(x1)), int(p.value(x2)), int(p.value(x3)), int(p.value(x4)) \
            , int(p.value(x5)), int(p.value(x6)), int(p.value(x7)), int(p.value(x8))], int(p.value(problem.objective))
        
magazine = []
coeffs, num_moves = integerMoves(s=start, d=end, v1=moves[0], v2=moves[1], v3 = moves[2],
         v4 = moves[3], v5 = moves[4], v6 = moves[5], v7 = moves[6], v8 = moves[7])
for i, cnt in enumerate(coeffs):
    for _ in range(cnt):
        magazine.append(moves[i])

        
def makePath():
    global magazine
    solved = False
    depth = 0
    while not solved and depth < 100:
        grid = createBoard()
        grid[start[0]][start[1]] = 0
        position = [start[0], start[1]]
        for index, e in enumerate(magazine):
            if checkValid(position, (e[1], e[0])):
                position[0] += e[1]
                position[1] += e[0]
                grid[position[0]][position[1]] = index + 1
            else:
                shuffle(magazine)
                depth += 1
                break
        if tuple(position) == end:
            solved = True
            print("It takes " + str(num_moves) + " moves")
            return grid
    depth = 0
    grid = createBoard()
    grid[start[0]][start[1]] = 0
    position = [start[0], start[1]]
    while not solved and depth < 100:
        depth += 1
        route_metrics = {}
        for i, x in enumerate(moves):
            if checkValid(position, (x[1], x[0])):
                temp = [position[0] + x[1], position[1] + x[0]]
                _ , metrics = integerMoves(s=temp, d=end, v1=moves[0], v2=moves[1], v3 = moves[2],\
                 v4 = moves[3], v5 = moves[4], v6 = moves[5], v7 = moves[6], v8 = moves[7])
                route_metrics[i] = metrics
        temp = min(route_metrics.values())
        index = [num for num, value in route_metrics.items() if value == temp]
        e = moves[index[0]]
        position[0] += e[1]
        position[1] += e[0]
        grid[position[0]][position[1]] = depth
        if tuple(position) == end:
            solved = True
            return grid

path = makePath()
printGrid(path)

Enter starting square: b2
Enter final square: a1
Move: (1, 2)
Move: (-2, -1)
Move: (1, 2)
Move: (-2, -1)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (-2, -1)
Move: (-2, -1)
Move: (-2, -1)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (-2, -1)
Move: (-2, -1)
Move: (1, 2)
Move: (1, 2)
Move: (-2, -1)
Move: (1, 2)
Move: (-2, -1)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (-2, -1)
Move: (-2, -1)
Move: (1, 2)
Move: (-2, -1)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (-2, -1)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (-2, -1)
Move: (-2, -1)
Move: (1, 2)
Move: (1, 2)
Move: (-2, -1)
Move: (-2, -1)
Move: (-2, -1)
Move: (-2, -1)
Move: (-2, -1)
Move: (1, 2)
Move: (1, 2)
Move: (-2, -1)
Move: (-2, -1)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (-2, -1)
Move: (1, 2)
Move: (1, 2)
Move: (-2, -1)
Move: (-2, -1)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (1, 2)
Move: (-2, -1)
Mo